In [6]:
# Pkg.add("PyCall")

LoadError: UndefVarError: Pkg not defined

In [ ]:
using PyCall
PyCall.libpython

"C:\\Users\\DELL\\.julia\\conda\\3\\python39.dll"

In [ ]:
a = pyimport("xcs")

PyObject <module 'xcs' from 'C:\\Users\\DELL\\.julia\\conda\\3\\lib\\site-packages\\xcs\\__init__.py'>

In [ ]:
ScenarioObserver=a.scenarios.ScenarioObserver 
MUXProblem=a.scenarios.MUXProblem
PreClassifiedData=a.scenarios.PreClassifiedData
XCSAlgorithm=a.XCSAlgorithm
BitString=a.bitstrings
Scenario=a.scenarios 

PyObject <module 'xcs.scenarios' from 'C:\\Users\\DELL\\.julia\\conda\\3\\lib\\site-packages\\xcs\\scenarios.py'>

In [ ]:
algorithm = XCSAlgorithm()


PyObject <xcs.XCSAlgorithm object at 0x0000000062223970>

In [ ]:
algorithm.exploration_probability = .1
algorithm.discount_factor = 0
algorithm.do_ga_subsumption = true
algorithm.do_action_set_subsumption = true

true

In [ ]:
py"""
def outputmodel(x):
    import logging
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    logger.info('Classifiers:\n\n%s\n', x)

"""

In [ ]:
using ZipFile, CSV, DataFrames, Random, StatsBase , Plots, Statistics

# read file from zip archive

z = ZipFile.Reader("results.zip")

# identify the right file in zip

# The diabetes dataset I found through kaggle have done split into 2 classifiers and select 50 percents of result from each 
# label. So i think there is more work in my dataset

a_file_in_zip = filter(x->x.name == "diabetes_binary_5050split_health_indicators_BRFSS2015.csv", z.files)[1]

#avoid changing the original files in the zip file. However, the dataset will not change but whatever.

a_copy = CSV.File(a_file_in_zip) |> DataFrame

close(z)

#show the dataset

a_copy

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,1.0,1.0,25.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0
3,0.0,1.0,1.0,1.0,29.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0,26.0,1.0,0.0,0.0
5,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0
6,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
7,0.0,1.0,1.0,1.0,36.0,0.0,0.0,0.0
8,0.0,0.0,0.0,1.0,26.0,1.0,0.0,0.0
9,0.0,0.0,0.0,1.0,21.0,0.0,0.0,0.0


In [ ]:
# Transfer DataFrame to matrix form

df=a_copy|>Tables.matrix

# Transfer the dataset to 2-classifiers. df_0 represents the result is 0, df_1 represents the result is 1.
# Due to my datasset is binary problems, and each result is 50 percents of the whole dataset. So i didn't add any other pre-actions for dataset. 

df_0 = df[df[:,1] .== 0, :]
df_1 = df[df[:,1] .== 1, :]

35346×22 Matrix{Float64}:
 1.0  1.0  1.0  1.0  30.0  1.0  0.0  1.0  …  30.0  1.0  0.0   9.0  5.0  1.0
 1.0  0.0  0.0  1.0  25.0  1.0  0.0  0.0      0.0  0.0  1.0  13.0  6.0  8.0
 1.0  1.0  1.0  1.0  28.0  0.0  0.0  0.0      0.0  1.0  0.0  11.0  4.0  6.0
 1.0  0.0  0.0  1.0  23.0  1.0  0.0  0.0      0.0  0.0  1.0   7.0  5.0  6.0
 1.0  1.0  0.0  1.0  27.0  0.0  0.0  0.0      0.0  0.0  0.0  13.0  5.0  4.0
 1.0  1.0  1.0  1.0  37.0  1.0  1.0  1.0  …   0.0  1.0  1.0  10.0  6.0  5.0
 1.0  1.0  1.0  1.0  28.0  1.0  0.0  1.0      0.0  0.0  1.0  12.0  2.0  4.0
 1.0  1.0  1.0  1.0  27.0  1.0  0.0  0.0     20.0  1.0  0.0   8.0  4.0  7.0
 1.0  1.0  1.0  1.0  34.0  1.0  1.0  0.0      7.0  1.0  0.0   9.0  5.0  4.0
 1.0  1.0  1.0  1.0  24.0  1.0  0.0  0.0      0.0  0.0  0.0  12.0  3.0  3.0
 1.0  1.0  0.0  1.0  31.0  0.0  0.0  0.0  …   5.0  0.0  0.0  13.0  4.0  4.0
 1.0  1.0  1.0  1.0  33.0  1.0  0.0  0.0     30.0  1.0  0.0  11.0  4.0  2.0
 1.0  1.0  1.0  1.0  27.0  1.0  0.0  0.0     30.0  1.0  0.0  1

In [ ]:
# The columns of matrix is 35346(here just using 35000 is the same), using random sub-sequence to select 70 percents of 
# columns as the train data.

sample = randsubseq(1:35000, 0.7)
train_df = vcat(df_0[sample, :], df_1[sample, :])

# Then from the not selected columns (which is 30 percents) to select the test data.

notsample = [i for i in 1:35000 if isempty(searchsorted(sample, i))]
test_df = vcat(df_0[notsample, :], df_1[notsample, :])

21226×22 Matrix{Float64}:
 0.0  0.0  0.0  0.0  25.0  1.0  0.0  0.0  …   0.0  0.0  0.0  11.0  6.0  8.0
 0.0  1.0  1.0  1.0  29.0  1.0  0.0  0.0      1.0  0.0  1.0  13.0  5.0  8.0
 0.0  0.0  0.0  1.0  29.0  1.0  0.0  0.0      0.0  0.0  1.0   4.0  4.0  4.0
 0.0  1.0  1.0  1.0  36.0  0.0  0.0  0.0      2.0  0.0  1.0  11.0  6.0  8.0
 0.0  0.0  0.0  1.0  26.0  1.0  0.0  0.0     15.0  0.0  0.0   7.0  5.0  8.0
 0.0  1.0  1.0  1.0  23.0  1.0  0.0  1.0  …   0.0  0.0  0.0   6.0  4.0  6.0
 0.0  1.0  1.0  1.0  25.0  1.0  0.0  0.0      0.0  0.0  1.0  11.0  6.0  8.0
 0.0  0.0  0.0  1.0  35.0  0.0  0.0  0.0      0.0  0.0  1.0   7.0  4.0  7.0
 0.0  0.0  0.0  1.0  21.0  0.0  0.0  0.0      0.0  0.0  0.0   3.0  4.0  6.0
 0.0  1.0  1.0  1.0  27.0  0.0  0.0  0.0      0.0  0.0  0.0  11.0  4.0  3.0
 0.0  0.0  0.0  1.0  21.0  0.0  0.0  0.0  …   0.0  0.0  0.0  11.0  5.0  7.0
 0.0  1.0  0.0  1.0  92.0  0.0  0.0  0.0      0.0  1.0  0.0   7.0  6.0  5.0
 0.0  0.0  0.0  1.0  27.0  0.0  0.0  0.0      1.0  0.0  0.0   

In [ ]:
# Divide the columns into features and result. From my dataset, From 2 to 22 columns are the attributes of whether diabetes or not.

X_train = train_df[:, 2:22]
X_test = test_df[:, 2:22]

y_train = train_df[:, 1]
y_test = test_df[:, 1]

21226-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [ ]:
using Distributions

# Transfer the X_train to 1 diminension.

dt = fit(ZScoreTransform, X_train, dims=1)

# Using StateBase package to transfer X_train and X_test(make them standard) to the same formate as dt.

X_train_std = StatsBase.transform(dt, X_train)
X_test_std = StatsBase.transform(dt, X_test)

# Transpose the X_train_std and X_test_std.
    
X_train_std, X_test_std= transpose(X_train_std), transpose(X_test_std)

([-1.145797733050531 -1.145797733050531 … 0.8727364947832692 0.8727364947832692; 0.9490452484555988 -1.0536689361233558 … -1.0536689361233558 -1.0536689361233558; … ; 0.07620955905525127 1.0482401640136292 … -1.8678516508615044 -0.8958210459031265; 0.5946181844448918 1.0547322110998474 … -0.7857238955199749 -0.7857238955199749], [-1.145797733050531 0.8727364947832692 … 0.8727364947832692 0.8727364947832692; -1.0536689361233558 0.9490452484555988 … -1.0536689361233558 0.9490452484555988; … ; 1.0482401640136292 0.07620955905525127 … 0.07620955905525127 -0.8958210459031265; 1.0547322110998474 1.0547322110998474 … -1.705951948829886 -2.1660659754848415])

In [ ]:
scenario = PreClassifiedData(X_train, y_train)

PyObject <xcs.scenarios.PreClassifiedData object at 0x00000000567C3760>

In [ ]:
model = algorithm.new_model(scenario)


PyObject <xcs.ClassifierSet object at 0x0000000062223E80>

In [ ]:
model = algorithm.run(scenario)


PyObject <xcs.ClassifierSet object at 0x0000000065CFAEB0>

In [ ]:
py"outputmodel"(model)

INFO:__main__:Classifiers:

1#011##111#011000#111 => 1.0
    Time Stamp: 6055
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
#0010#01#1###11001#1# => 1.0
    Time Stamp: 6384
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
0#0110##011101#0#111# => 1.0
    Time Stamp: 6670
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
000#0001#1011111#011# => 1.0
    Time Stamp: 6944
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
001#111000#1011100### => 1.0
    Time Stamp: 7223
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
011#10001##00101##111 => 1.0
    Time Stamp: 7417
    Average Reward: 1e-05
    Error: 1e-05
    Fitness: 1e-05
    E

In [ ]:
print(length(model))

18